In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Import data set

In [ ]:
import pandas as pd

In [ ]:
data = pd.read_csv("../input/credit-card-customers/BankChurners.csv") 
data.head()

In [ ]:
data.drop(['Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1'], inplace=True, axis=1)
data.drop(['Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2'], inplace=True, axis=1)

In [ ]:
data.columns

In [ ]:
data.head()

# Encode

In [ ]:
data['Income_Category'].unique()

In [ ]:
data = data.replace({'Unknown': 0 , 'Less than $40K':1, '$40K - $60K':2, 
                     '$80K - $120K':3, '$60K - $80K':4, '$120K +':5})

In [ ]:
data['Income_Category'].unique()

In [ ]:
data['Attrition_Flag'].unique()

In [ ]:
data = data.replace({'Existing Customer':0, 'Attrited Customer':1})

In [ ]:
data['Attrition_Flag'].unique()

In [ ]:
data['Gender'].unique()

In [ ]:
data = data.replace({'M':0, 'F':1})
data['Gender'].unique()

In [ ]:
data['Education_Level'].unique()

In [ ]:
data = data.replace({'High School':1, 'Graduate':2, 'Uneducated':3,'College':4,'Post-Graduate':5,'Doctorate':6})
data['Education_Level'].unique()

In [ ]:
data['Marital_Status'].unique()

In [ ]:
data = data.replace({'Married':1, 'Single':2, 'Divorced':3})
data['Marital_Status'].unique()

In [ ]:
data['Card_Category'].unique()

In [ ]:
data = data.replace({'Blue':0, 'Gold':1, 'Silver':2, 'Platinum':3})
data['Card_Category'].unique()

In [ ]:
import seaborn as sns
import plotly.express as ex


In [ ]:
fig = ex.pie(data, names='Attrition_Flag', title = 'Chunc clients and not chunc clients'  )
fig.show()

imbalance data but I will do it without sampling

# Feature selection

In [ ]:
data.columns

# Find Importance Feature

Correlation Matrix with Heatmap

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.figure(figsize=(20,20))
cor = data.corr()
sns.heatmap(cor, annot=True, cmap=plt.cm.Reds)
plt.show()

In [ ]:
cor_target = abs(cor["Attrition_Flag"])
relevant_features = cor_target
relevant_features

using a Tree-Based Classifiers

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier
import matplotlib.pyplot as plt


In [ ]:
X = pd.get_dummies(data.drop(columns=['Attrition_Flag']))
y=data['Attrition_Flag'] 

model = ExtraTreesClassifier()
#you can change Feature Importance
model.fit(X,y)
print(model.feature_importances_) 

feat_importances = pd.Series(model.feature_importances_, index=X.columns)
feat_importances.nlargest(10).plot(kind='barh')
#select top 10 feature
plt.show()

In [ ]:
feat_importances.nlargest(10)

# Train Test Split

In [ ]:
X = data[['Total_Trans_Ct','Total_Trans_Amt','Total_Revolving_Bal','Total_Ct_Chng_Q4_Q1','Total_Relationship_Count','Avg_Utilization_Ratio','Contacts_Count_12_mon','Total_Amt_Chng_Q4_Q1','Months_Inactive_12_mon','Avg_Open_To_Buy']]
X

In [ ]:
y=data['Attrition_Flag'] 
y

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=20,shuffle=True)

# Random Forest Classifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report, confusion_matrix
from sklearn import metrics

In [ ]:
rf=RandomForestClassifier()
rf.fit(X_train,y_train)
print('the accuracy of Test data is:',accuracy_score(y_test,rf.predict(X_test))*100,'%')

In [ ]:
print(confusion_matrix(y_test,rf.predict(X_test)))
print(classification_report(y_test,rf.predict(X_test)))

In [ ]:
cf=metrics.confusion_matrix(y_test,rf.predict(X_test), labels=[1,0])
sns.heatmap(cf,annot=True)

# Sampling data

find total 1 and 0

In [ ]:
one=0
zero=0
for i in data['Attrition_Flag']:
    if i==1:
        one+=1
    else:
        zero+=1
print('Total 1:',one)
print('Total 0:',zero)


# Try under sampling with min value 1627

In [ ]:
data_under = []
for toxic_type in ['Attrition_Flag']:
    df_class_0 = data[data['Attrition_Flag'] == 0]
    df_class_1 = data[data['Attrition_Flag'] == 1]

    df_class_0_under = df_class_0.sample(1627)
    df_class_1_under = df_class_1.sample(1627)
    
    data_under.append(df_class_0_under)
    data_under.append(df_class_1_under)
          
df_train_under = pd.concat(data_under, axis=0,ignore_index=True)
print(df_train_under.shape)

In [ ]:
df_train_under

In [ ]:
sns.countplot(df_train_under['Attrition_Flag'])
plt.title('Target repartition on training data')
plt.show()

In [ ]:
X = df_train_under[['Total_Trans_Ct','Total_Trans_Amt','Total_Revolving_Bal','Total_Ct_Chng_Q4_Q1','Total_Relationship_Count','Avg_Utilization_Ratio','Contacts_Count_12_mon','Total_Amt_Chng_Q4_Q1','Months_Inactive_12_mon','Avg_Open_To_Buy']]
y=df_train_under['Attrition_Flag'] 
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=20,shuffle=True)

In [ ]:
rf_under=RandomForestClassifier()
rf_under.fit(X_train,y_train)
y_pred_under=rf_under.predict(X_test)
print('the accuracy of Test undersampling data is:',accuracy_score(y_test,y_pred_under)*100,'%')

In [ ]:
print(confusion_matrix(y_test,y_pred_under))
print(classification_report(y_test,y_pred_under))

In [ ]:
cf=metrics.confusion_matrix(y_test,y_pred_under, labels=[1,0])
sns.heatmap(cf,annot=True)